In [1]:
import ipywidgets as widgets
from IPython.display import display
from datetime import date, datetime, timedelta

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors

from scipy.stats import linregress


In [2]:
from_date = date(1900, 1, 1) 
to_date = date.today()
to_date = date(2021, 9, 10) 

portfolio_Beta = pd.DataFrame()

symbols = ['^GSPC','FLR']

# Build Portfolio prices, find oldest common date and remove rows with no data.           
for ind,symb in enumerate(symbols):
    tmp = yf.download(symb, from_date, to_date, interval="1wk")    
    tmp = tmp.loc[:, "Adj Close":"Adj Close"]              # Remove columns except Adj Close    
    tmp.rename(columns={"Adj Close":symb}, inplace=True)    # Rename columns
    
    # Sort values in descending order, NA values at the bottom
    tmp.sort_values(by='Date', ascending=True, na_position='last',inplace=True)     
    
    if ind==0:
        portfolio_Beta = tmp
        continue
    
    closest_date = max(tmp.index[0],portfolio_Beta.index[0])   # Determine closest date - the oldest date for which there is information for both Stocks
    print(closest_date)    
    tmp = tmp.loc[tmp.index >= closest_date]                  # Remove rows of data earlier than closest date
    portfolio_Beta = portfolio_Beta.loc[portfolio_Beta.index >= closest_date]
    portfolio_Beta = portfolio_Beta.join(tmp)                   # Merge dataframes now that they have the same indexes

# Calculate and Replace columns of prices in portfolio with Returns
for col in portfolio_Beta.columns:
    portfolio_Beta[col] = (portfolio_Beta[col]/portfolio_Beta[col].shift(1))-1

# RESAMPLE to specific day if needed
# portfolio_Beta = portfolio_Beta.resample("W-THU").first()

# Sort values in descending order, NA values at the bottom
portfolio_Beta.sort_values(by='Date', ascending=False, na_position='last',inplace=True)
portfolio_Beta = portfolio_Beta.drop(portfolio_Beta.index[-1])
portfolio_Beta

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

2000-11-27 00:00:00


,^GSPC,FLR
Date,,
2021-09-06,-0.016944,-0.040680
2021-08-30,0.005779,-0.014363
2021-08-23,0.015242,0.060952
2021-08-16,-0.005893,-0.094828
2021-08-09,0.007096,0.028977
...,...,...
2001-01-01,-0.016610,0.017014
2000-12-25,0.010973,0.013409
2000-12-18,-0.004725,0.105933


In [3]:

# Dynamic selector for Beta estimation
def beta_yr(n):  
    start_indx = n*52

    x = portfolio_Beta.iloc[0:start_indx]['^GSPC']
    y = portfolio_Beta.iloc[0:start_indx]['FLR'] 
    
    # Perform linear regression
    slope, intercept, r_value, p_value, std_err = linregress(x, y)       

    # Plotting Scattered data and best-fit line 
    plt.figure(figsize=(14, 5))
    plt.scatter(x, y)
    plt.plot(x, slope * x + intercept, color='black', label='Best Fit Line')
    
    plt.title('-vs-'.join(symbols))
    plt.xlabel(symbols[0])
    plt.ylabel(symbols[1])
    plt.grid(True)
    plt.show()
    print('Beta-' + str(n) +'yrs: '+str(slope))

# Create RadioButtons widget
n_years = widgets.RadioButtons(
    options=[1,2,3,5],
    description='Select number of years:',
    disabled=False
)

# Create the interactive plot
interactive_plot = widgets.interactive(
    beta_yr,
    n=n_years
)

# Display the interactive plot
display(interactive_plot)

interactive(children=(RadioButtons(description='Select number of years:', options=(1, 2, 3, 5), value=1), Outp…

In [4]:

# Dynamic selector for Rolling Beta
def beta_rolling(years):  
    roll_Beta_arr = pd.DataFrame(0, index=portfolio_Beta[:-52*years[0]].index, columns=['Beta-'+str(years[0])+'yrs','Beta-'+str(years[1])+'yrs'], dtype=float)

    for i in range(len(portfolio_Beta)-52*years[1]):
        x = portfolio_Beta.iloc[i:i+52*years[1]]['^GSPC']
        y = portfolio_Beta.iloc[i:i+52*years[1]]['FLR'] 
        slope, intercept, r_value, p_value, std_err = linregress(x, y)    

        roll_Beta_arr.iloc[i]['Beta-'+str(years[1])+'yrs'] = slope
        roll_Beta_arr = roll_Beta_arr.replace(0, np.nan)     # Replace zeros with NaN
        
    for i in range(len(portfolio_Beta)-52*years[0]):
        x = portfolio_Beta.iloc[i:i+52*years[0]]['^GSPC']
        y = portfolio_Beta.iloc[i:i+52*years[0]]['FLR'] 
        slope, intercept, r_value, p_value, std_err = linregress(x, y)    

        roll_Beta_arr.iloc[i]['Beta-'+str(years[0])+'yrs'] = slope        

    # Plotting Scattered data and best-fit line 
    plt.figure(figsize=(14, 5))
    plt.plot(roll_Beta_arr.index,roll_Beta_arr['Beta-'+str(years[0])+'yrs'], linewidth=1.5, color='black', label='Beta-'+str(years[0])+'yrs')
    plt.plot(roll_Beta_arr.index,roll_Beta_arr['Beta-'+str(years[1])+'yrs'], linewidth=2.5, color='blue', label='Beta-'+str(years[1])+'yrs')
    
    plt.title("Rolling Beta "+str(years[0])+'vs'+str(years[1])+' yrs')
    plt.xlabel('Date')
    plt.ylabel('Beta')
    plt.legend()
    plt.grid(True)
    plt.show()

n_years_widget = widgets.IntRangeSlider(
    value=[1, 3],
    min=1,
    max=5,
    step=1,
    description='Years:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
)
    
# Display the interactive plot
rolBeta_controls = widgets.interactive(beta_rolling, years=n_years_widget)
display(rolBeta_controls)


interactive(children=(IntRangeSlider(value=(1, 3), continuous_update=False, description='Years:', max=5, min=1…

In [5]:
from_date = date(1900, 1, 1) 
to_date = date.today()

portfolio = pd.DataFrame()

symbols = ['LVS','BBY','LEN','SPLK','VTRS','XPO','MOS','OLED','TTEC','WING']

# Build Portfolio prices, find oldest common date and remove rows with no data.           
for ind,symb in enumerate(symbols):
    tmp = yf.download(symb, from_date, to_date, interval="1wk")    
    tmp = tmp.loc[:, "Adj Close":"Adj Close"]               # Remove columns except Adj Close    
    tmp.rename(columns={"Adj Close":symb}, inplace=True)    # Rename columns
    
    # Sort values in descending order, NA values at the bottom
    tmp.sort_values(by='Date', ascending=True, na_position='last',inplace=True)     
    
    if ind==0:
        portfolio = tmp
        continue
    
    closest_date = max(tmp.index[0],portfolio.index[0])    # Determine closest date - the oldest date for which there is information for both Stocks
    tmp = tmp.loc[tmp.index >= closest_date]               # Remove rows of data earlier than closest date
    portfolio = portfolio.loc[portfolio.index >= closest_date]
    portfolio = portfolio.join(tmp)                        # Merge dataframes now that they have the same indexes

# Calculate and Replace columns of prices in portfolio with Returns
for col in portfolio.columns:
    portfolio[col] = (portfolio[col]/portfolio[col].shift(1))-1

# RESAMPLE to specific day if needed
# portfolio = portfolio.resample("W-THU").first()

# Sort values in descending order, NA values at the bottom
portfolio.sort_values(by='Date', ascending=False, na_position='last',inplace=True)
portfolio = portfolio.drop(portfolio.index[-1])
portfolio


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,LVS,BBY,LEN,SPLK,VTRS,XPO,MOS,OLED,TTEC,WING
Date,,,,,,,,,,
2024-03-25,0.014970,-0.007670,-0.006123,NaN,-0.020270,-0.020332,-0.021809,-0.007763,-0.047094,0.032838
2024-03-18,-0.018225,0.046655,0.063933,0.000000,-0.008375,0.022501,-0.028660,0.043597,-0.091075,0.024881
2024-03-11,-0.009319,-0.007127,-0.046410,0.002876,-0.016530,0.001879,0.023021,-0.056787,-0.060736,-0.017325
2024-03-04,0.004485,0.008470,0.016782,0.001857,-0.026211,0.015174,-0.001265,-0.031200,-0.157174,-0.033554
2024-02-26,-0.063379,0.021500,0.041269,0.003922,-0.069475,-0.010583,0.004765,0.016231,-0.233278,0.081014
...,...,...,...,...,...,...,...,...,...,...
2015-07-13,-0.010394,-0.030946,-0.015366,0.014477,-0.038613,0.045547,0.004884,0.003956,0.000734,-0.107912
2015-07-06,0.018952,0.012836,0.041015,0.030407,0.015687,-0.003147,-0.034512,-0.049485,0.009259,0.205469
2015-06-29,0.051172,-0.013836,-0.015006,-0.009378,0.015349,-0.027328,0.002148,-0.040091,-0.013518,0.009558


In [6]:
init_total_GE = 100000
Total_GE = init_total_GE

# Portfolio Summary by Shares and initial capital
portfolio_summary = pd.DataFrame()
portfolio_summary.index = symbols
portfolio_summary['curr_price'] = [portfolio.iloc[0][sym] for sym in portfolio.columns]
portfolio_summary['beta'] = [0]*len(symbols)
portfolio_summary['#shares'] = [0]*len(symbols)
portfolio_summary['long/short'] = [0]*len(symbols)

def portf_update():
    portfolio_summary['gross_exposure'] = abs(portfolio_summary['#shares']*portfolio_summary['curr_price'])
    portfolio_summary['net_exposure'] = portfolio_summary['long/short']*portfolio_summary['gross_exposure']
    portfolio_summary['net_weight'] = portfolio_summary['net_exposure']/sum(portfolio_summary['gross_exposure'])
    portfolio_summary['net_weighted_Beta'] = portfolio_summary['beta']*portfolio_summary['net_weight']
    
# Function to calculate sum of all IntText values
def exposure_chng(change):
    global Total_GE 
    portfolio_summary.loc[change['owner'].description,'#shares'] = round((change['new']/portfolio_summary.loc[change['owner'].description]['curr_price']),0)
    Total_GE = Total_GE + change['new'] - change['old']
    portf_update()
    print('New Total GE: '+ str(Total_GE))
     
# Function to handle radio button changes
def position_chng(change):
    portfolio_summary.loc[change['owner'].description,'long/short'] = -1 if change['owner'].value == 'short' else 1
    portf_update()
    print('New position '+ change['owner'].description+': '+change['owner'].value)

# Create widgets for each symbol
positions = []
exposures = []
for symbol in symbols:
    exp = widgets.IntText(description=symbol, value=init_total_GE/len(symbols), layout=widgets.Layout(margin='0px 0px 0px -40px', padding='0px 0px 0px 0px',width='150px'))
    pos = widgets.RadioButtons(options=['long', 'short'], description=symbol, layout=widgets.Layout(margin='0px 0px 0px -40px', padding='0px',width='auto'))
    portfolio_summary.loc[symbol,'long/short'] = -1 if pos.value == 'short' else 1
    portfolio_summary.loc[symbol,'#shares'] = round((exp.value/abs(portfolio_summary.loc[symbol,'curr_price'])),0)
    positions.append(pos)
    exposures.append(exp)

portf_update()
Total_GE = sum(portfolio_summary['gross_exposure'])

# Display widgets
widgets_box = widgets.HBox([widgets.VBox([exp, pos],layout=widgets.Layout(margin='0px', padding='0px',width='auto')) for exp, pos in zip(exposures, positions)], layout=widgets.Layout(margin='0px', padding='0px', width='1100px'))
display(widgets_box)

# Register the calculate_sum function to be called whenever the value of any IntText widget changes
for exp,pos in zip(exposures,positions):
    exp.observe(exposure_chng, 'value')
    pos.observe(position_chng, 'value')


In [7]:
portfolio_summary

,curr_price,beta,#shares,long/short,gross_exposure,net_exposure,net_weight,net_weighted_Beta
LVS,0.014970,0,668000.0,1,10000.000305,10000.000305,NaN,NaN
BBY,-0.007670,0,1303816.0,1,10000.000047,10000.000047,NaN,NaN
LEN,-0.006123,0,1633130.0,1,9999.997354,9999.997354,NaN,NaN
SPLK,NaN,0,NaN,1,NaN,NaN,NaN,NaN
VTRS,-0.020270,0,493334.0,1,10000.003848,10000.003848,NaN,NaN
XPO,-0.020332,0,491844.0,1,9999.999736,9999.999736,NaN,NaN
MOS,-0.021809,0,458529.0,1,9999.995410,9999.995410,NaN,NaN
OLED,-0.007763,0,1288146.0,1,9999.998254,9999.998254,NaN,NaN
TTEC,-0.047094,0,212341.0,1,10000.012903,10000.012903,NaN,NaN
WING,0.032838,0,304530.0,1,10000.013583,10000.013583,NaN,NaN
